In [1]:
import xarray as xr 
import clix
import pandas as pd

INFO:root:Using clix cache dir at: /tmp/dask_
INFO:root:Using dask cache dir at: /tmp/dask_
/Users/aidanstarr/miniforge3/envs/productivity_loss_v2/lib/python3.10/site-packages/clix/aggregation_interpolation.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/Users/aidanstarr/miniforge3/envs/productivity_loss_v2/lib/python3.10/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
from dask.distributed import Client

client = Client()

import s3fs, dask
fs = s3fs.S3FileSystem(anon=False, use_listings_cache=False, asynchronous=True)
dask.config.set(
    {"s3.multipart_chunksize": "50MB"})

/Users/aidanstarr/miniforge3/envs/productivity_loss_v2/lib/python3.10/site-packages/bokeh/core/property/primitive.py:37: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  bokeh_bool_types += (np.bool8,)
INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:64351
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:64354'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:64355'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:64356'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:64357'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:64358'
INF

In [13]:
fn = '../data/sample_assets.csv'
loss_function = 'HOTHAPS'


## read csv  
df_in = pd.read_csv(fn)[['asset_id','latitude','longitude','asset_type']]
df_in = df_in.drop_duplicates('asset_id').rename(columns={'latitude':'lat','longitude':'lon'})
## load asset map 
asset_map = pd.read_csv('../src/asset_map.csv')

## load aircon
aircon = xr.open_zarr('s3://hazard-science-data/productivity_loss_v2/aircon/AirCon_SSPs.zarr/') 
aircon = aircon.sel(SSP='2.0').isel(year=0)['ac_penetration']

## load losses
ds_dict = {}
for scenario in ['ssp126','ssp245','ssp370','ssp585']:
    ds_dict[scenario] = {}
    for intensity in ['low','moderate','high']:
        ds_dict[scenario][intensity] = xr.open_zarr(f"s3://hazard-science-data/productivity_loss_v2/climate_outputs/projections_corrected/{scenario}/CMIP6-ScenarioMIP_{loss_function}_productivity_loss_{intensity}_{scenario}.zarr.zarr/")

In [53]:
## output
df_out = df_in.copy(deep=True).set_index('asset_id')
df_out['work_intensity'] = df_out['asset_type'].map(asset_map.set_index('asset_type').to_dict()['intensity'])
df_out=df_out.reset_index()
## sample
df_out_ssp126 = df_out.copy(deep=True)


In [40]:
df_out.head().to_clipboard()

In [25]:
ds_out = extract_asset_data(ds_dict['ssp126']['high'],df_out_ssp126)
ds_out['median'].to_dataframe()

lat         lon  \
year asset_id                                                                  
2025 1203 - BP INTERNATIONAL LIMITED                   51.734730    5.494284   
     19238 - BP INTERNATIONAL LIMITED                  51.935600    4.125400   
     21 - DP WORLD LIMITED                             51.550000    4.460072   
     5420 - BP INTERNATIONAL LIMITED                   51.914513    4.546143   
     72 - ABU DHABI NATIONAL ENERGY COMPANY - P.J.S.C  24.090403   52.652494   
...                                                          ...         ...   
2100 74 - SIEMENS ENERGY LIMITED                        4.904735  114.920770   
     450 - MINOR INTERNATIONAL PUBLIC COMPANY LIMITED   4.905742  114.916441   
     1572 - SAMSUNG C AND T CORPORATION                47.874492  106.915138   
     1689 - SAMSUNG C AND T CORPORATION                47.920030  106.891174   
     1223 - SAMSUNG C AND T CORPORATION                42.842570   74.604920   

                                                          median  
year asset_id                                                     
2025 1203 - BP INTERNATIONAL LIMITED                    0.034642  
     19238 - BP INTERNATIONAL LIMITED                   0.022831  
     21 - DP WORLD LIMITED                              0.022831  
     5420 - BP INTERNATIONAL LIMITED                    0.022831  
     72 - ABU DHABI NATIONAL ENERGY COMPANY - P.J.S.C  12.984390  
...                                                          ...  
2100 74 - SIEMENS ENERGY LIMITED                       11.362954  
     450 - MINOR INTERNATIONAL PUBLIC COMPANY LIMITED  11.266126  
     1572 - SAMSUNG C AND T CORPORATION                 0.022831  
     1689 - SAMSUNG C AND T CORPORATION                 0.022831  
     1223 - SAMSUNG C AND T CORPORATION                 0.114466  

[4961 rows x 3 columns]

In [50]:
import numpy as np  

for year in ds_dict['ssp126']['high'].year.values:
    for newcol in ['median','minimum','maximum']:
        df_out_ssp126[newcol+"_"+str(year)] = np.full_like(df_out_ssp126['lat'],np.nan)

for i in df_out_ssp126.index:
    wi = df_out_ssp126.loc[i,'work_intensity']
    lat = df_out_ssp126.loc[i,'lat']
    lon = df_out_ssp126.loc[i,'lon']
    for year in ds_dict['ssp126']['high'].year.values:
        for newcol in ['median','minimum','maximum']:
            df_out_ssp126.loc[i,newcol+"_"+str(year)] = ds_dict['ssp126'][wi].sel(lat=lat,lon=lon,year=year,method='nearest')[newcol]

ValueError: setting an array element with a sequence.

In [65]:
df_out_ssp126 = df_out_ssp126.loc[:100,:]

In [69]:
years = ds_dict['ssp126']['high'].year.values
newcols = ['median', 'minimum', 'maximum']

# Create new columns in one operation
col_names = [f"{year}_{stat}" for year in years for stat in newcols]
df_out_ssp126 = df_out_ssp126.assign(**{col: np.nan for col in col_names})

# function to extract data from xarray
def extract_values(row):
    """Fetches nearest productivity loss values for given lat, lon, and work intensity."""
    ds = ds_dict['ssp126'][row['work_intensity']]
    result = {f"{year}_{stat}": np.round(ds.sel(lat=row['lat'], lon=row['lon'], year=year, method='nearest')[stat].values,4)
              for stat in newcols for year in years }
    return pd.Series(result)

# apply function across all rows
df_out_ssp126[col_names] = df_out_ssp126.apply(extract_values, axis=1)

Debugging information
---------------------
old task state: released
old run_spec: (<function execute_task at 0x312b13a30>, (ImplicitToExplicitIndexingAdapter(array=CopyOnWriteArray(array=LazilyIndexedArray(array=<xarray.backends.zarr.ZarrArrayWrapper object at 0x380dfb4c0>, key=BasicIndexer((slice(None, None, None), slice(None, None, None), slice(None, None, None)))))),), {})
new run_spec: (<function execute_task at 0x312b13a30>, (ImplicitToExplicitIndexingAdapter(array=CopyOnWriteArray(array=LazilyIndexedArray(array=<xarray.backends.zarr.ZarrArrayWrapper object at 0x380dedc40>, key=BasicIndexer((slice(None, None, None), slice(None, None, None), slice(None, None, None)))))),), {})
old token: ('tuple', (('80935a1067ef908b', []), ('tuple', (('c961c87336555955', ['eb55f6447623a5c5']),)), ('dict', ())))
new token: ('tuple', (('80935a1067ef908b', []), ('tuple', (('c961c87336555955', ['34c96acdcadb1bbb']),)), ('dict', ())))
old dependencies: set()
new dependencies: set()

Debugging informat

In [22]:
def extract_asset_data(ds, asset_df, lat_name="lat", lon_name="lon", time_name="year"):
    """
    Extracts values from a gridded xarray dataset at specific asset locations.
    
    Parameters:
        ds (xr.Dataset): Gridded dataset with dimensions (time, lat, lon).
        asset_df (pd.DataFrame): DataFrame with 'asset_id', 'lat', and 'lon'.
        lat_name (str): Name of latitude dimension in dataset (default: "lat").
        lon_name (str): Name of longitude dimension in dataset (default: "lon").
        time_name (str): Name of time dimension in dataset (default: "year").
    
    Returns:
        xr.Dataset: Dataset with dimensions (asset_id, year) containing extracted values.
    """
    # Convert asset locations into xarray DataArray
    asset_lats = xr.DataArray(asset_df["lat"], dims="asset_id", coords={"asset_id": asset_df["asset_id"]})
    asset_lons = xr.DataArray(asset_df["lon"], dims="asset_id", coords={"asset_id": asset_df["asset_id"]})

    # Interpolate dataset values at asset locations
    extracted_data = ds.interp({lat_name: asset_lats, lon_name: asset_lons}, method="linear")

    # Reshape dataset to include asset_id, lat, lon
    extracted_data = extracted_data.assign_coords(
        lat=("asset_id", asset_df["lat"]),
        lon=("asset_id", asset_df["lon"])
    )

    return extracted_data